In [262]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    Article
)

from pikepdf import Pdf, PdfImage
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [263]:
url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
url = 'https://www.biorxiv.org/content/10.1101/456574v1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.12.21.521407v1?rss=1'
article = Article.from_content_page_url(url)

In [ ]:
ii = 8
article.merged_images[ii]['image']

In [249]:
sets_to_merge = pdf_extraction.identify_images_to_merge(article.images)

In [258]:
all_images_to_merge = []
for x in sets_to_merge:
    all_images_to_merge += x

In [259]:
# Remove the images that were merged from the list of images
pruned_images = [image for image in article.images if image not in all_images_to_merge]

In [254]:
# Merge the images in each set
merged_images = []
for set_to_merge in sets_to_merge:
    merged_images.append(pdf_extraction.merge_image_pieces(set_to_merge))

In [260]:
images = merged_images + pruned_images
    
# sort the images by page number and then by y0 position
images = sorted(images, key=lambda x: (x['page'], x['y0']))

In [ ]:
article.merged_images

In [103]:
import pandas as pd

In [125]:
df = pd.DataFrame(article.layout)

In [127]:
df.loc[df['text'].str.contains('fig', case=False).fillna(False), :]

,page,type,x0,y0,x1,y1,text
198,4,LTTextBoxHorizontal,72.02500,87.4330,541.38400,244.8060,Study Design & Experimental Subjects (Fig. 1) ...
684,8,LTTextBoxHorizontal,72.02500,308.5080,542.22100,447.2880,An exploratory mediation analysis was also per...
1589,11,LTTextBoxHorizontal,72.02500,186.4580,531.37200,277.7380,Higher CFI scores in old versus young dogs \nO...
1590,11,LTTextBoxHorizontal,72.02500,74.6110,540.68800,150.7080,CFI scores increased with age in older dogs bu...
1890,13,LTTextBoxHorizontal,72.02500,79.6830,542.41200,202.7080,Higher HRQL scores in old versus young dogs \n...
1959,14,LTTextBoxHorizontal,72.02500,642.6080,528.62800,718.1280,0.001). Pairwise Wilcoxon rank sum tests demon...
1960,14,LTTextBoxHorizontal,72.02500,482.2360,542.12600,622.8580,HRQL scores declined with age in old dogs but ...
1962,14,LTTextBoxHorizontal,72.02500,109.6780,541.03600,318.8260,Clinically-assessed CFI is inversely related t...
1984,15,LTTextBoxHorizontal,72.02500,430.7880,542.17000,717.9460,Age-associated disease may mediate size differ...
2105,20,LTTextBoxHorizontal,72.02500,278.1440,242.85000,294.1440,Figure & Table Legends \n


In [138]:
df.query('type == "LTFigure"')

,page,type,x0,y0,x1,y1,text
2971,29,LTFigure,91.938980,396.00000,450.922080,638.91340,NaN
2986,30,LTFigure,4.418307,207.00980,571.581707,632.38230,NaN
3001,31,LTFigure,12.000000,320.00010,548.740200,698.95390,NaN
3018,32,LTFigure,93.992130,373.91740,443.974330,723.89960,NaN
3020,32,LTFigure,106.902600,11.72253,469.097500,373.91743,NaN
3039,33,LTFigure,64.080710,27.48029,511.919510,349.80119,NaN
3041,33,LTFigure,58.753930,355.47570,556.369930,713.62410,NaN
3060,34,LTFigure,84.495080,17.36868,509.134980,357.20468,NaN
3062,34,LTFigure,78.741140,381.04940,514.890540,708.16150,NaN
3079,35,LTFigure,17.673230,204.60310,569.673330,706.47040,NaN
